In [ ]:
"""
================================================================================
NOTEBOOK 03: RULE-BASED TEMPLATE EXPLANATION BASELINE
================================================================================
Copyright (c) 2026 Ajay Mahale. All Rights Reserved.
License: CC BY-NC-ND 4.0
https://creativecommons.org/licenses/by-nc-nd/4.0/


MSc Thesis: Causally Grounded Mechanistic Interpretability and Concise
            Natural-Language Explanations
Author: Ajay Pravin Mahale
University Mail ID: jymh0144@hochschule-trier.de
Personal Mail ID: Mahale.ajay01@gmail.com
Institution: Hochschule Trier
Supervisor: Prof. Dr. Ernst Georg Haffner

Purpose: Generate template-based natural language explanations as BASELINE.
         These are deterministic, rule-based mappings (NOT learned).

Note: This is the BASELINE method. See Notebook 08 for learned (LLM) method.
================================================================================
"""

# ==============================================================================
# CELL 1: Install Dependencies
# ==============================================================================

!pip install -q transformer-lens

print("Install done")


Install done


In [ ]:
# ==============================================================================
# CELL 2: REPRODUCIBILITY SETUP + IMPORTS
# ==============================================================================

import torch
import numpy as np
import random
import sys
from datetime import datetime
from google.colab import drive

drive.mount('/content/drive')

# Set seeds
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
print(f"✅ Seed set: {SEED}")

print("=" * 60)
print("NOTEBOOK 03: TEMPLATE EXPLANATION BASELINE")
print("=" * 60)
print(f"Random Seed:     {SEED}")
print(f"PyTorch:         {torch.__version__}")
print(f"CUDA Available:  {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU:             {torch.cuda.get_device_name(0)}")
print(f"Timestamp:       {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)


THESIS_DIR = '/content/drive/MyDrive/thesis'
EXPERIMENTS_DIR = f'{THESIS_DIR}/experiments'

from transformer_lens import HookedTransformer

# Load model
model = HookedTransformer.from_pretrained("gpt2-small")
model.cuda()

# Circuit results from Notebook 02 (IOI Reproduction)
# These are the 6 heads identified via activation patching
head_importance = {
    (9, 9): 0.174,   # Name Mover - highest importance
    (8, 10): 0.123,  # S-Inhibition
    (7, 3): 0.103,   # S-Inhibition
    (10, 6): 0.089,  # Backup
    (9, 6): 0.063,   # Name Mover
    (10, 0): 0.062,  # Name Mover
}

# Test prompts (same as Notebook 02)
test_prompts = [
    "When Mary and John went to the store, John gave a drink to",
    "When Alice and Bob went to the park, Bob handed a flower to",
    "When Sarah and Tom went to the office, Tom sent an email to",
    "When Emma and James went to the restaurant, James passed the menu to",
    "When Lisa and David went to the library, David gave a book to",
]

# Words to exclude from name detection
NOT_NAMES = {'When', 'The', 'Then', 'There', 'They', 'This', 'That', 'What', 'Where', 'Who', 'How'}

print("✅ Model loaded and setup complete")
print(f"   Device: {model.cfg.device}")
print(f"   Circuit heads: {len(head_importance)}")
print(f"   Test prompts: {len(test_prompts)}")


Mounted at /content/drive
✅ Seed set: 42
NOTEBOOK 03: TEMPLATE EXPLANATION BASELINE
Random Seed:     42
PyTorch:         2.9.0+cu126
CUDA Available:  True
GPU:             Tesla T4
Timestamp:       2026-02-06 12:02:42


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda
✅ Model loaded and setup complete
   Device: cuda
   Circuit heads: 6
   Test prompts: 5


In [ ]:
# ==============================================================================
# CELL 3: Natural Language Explanation Generator
# ==============================================================================
# OUR NOVEL CONTRIBUTION: Circuit → Human-Readable Explanation

# IOI circuit knowledge from our reproduction
ioi_circuit = {
    "Name Mover Heads": {
        "heads": [(9, 9), (9, 6), (10, 0)],
        "function": "Copy indirect object name to output position",
    },
    "S-Inhibition Heads": {
        "heads": [(7, 3), (8, 10), (8, 6)],
        "function": "Suppress the repeated subject name",
    },
}

def generate_nl_explanation(prompt, model, head_importance):
    """
    NOVEL CONTRIBUTION:
    Convert mechanistic circuit findings into natural language explanation.

    This function:
    1. Runs the model and extracts attention patterns
    2. Identifies names in the prompt
    3. Generates human-readable explanation of model behavior
    """

    tokens = model.to_tokens(prompt)
    logits, cache = model.run_with_cache(tokens)
    token_strs = model.to_str_tokens(prompt)

    # Get prediction
    top_token = logits[0, -1].argmax().item()
    predicted_word = model.to_string([top_token]).strip()
    confidence = torch.softmax(logits[0, -1], dim=-1)[top_token].item()

    # Find names (pos, name) tuples
    names = []
    for i, tok in enumerate(token_strs):
        tok_clean = tok.strip()
        if (tok_clean and tok_clean[0].isupper() and tok_clean.isalpha() and
            len(tok_clean) > 1 and tok_clean not in NOT_NAMES):
            if tok_clean not in [n[1] for n in names]:
                names.append((i, tok_clean))

    # Get attention from key heads to use in explanation
    key_heads = [(9, 9), (9, 6), (8, 10)]
    attention_info = {}

    for layer, head in key_heads:
        pattern = cache[f"blocks.{layer}.attn.hook_pattern"][0, head, -1]
        attention_info[(layer, head)] = {}
        for pos, name in names:
            attention_info[(layer, head)][name] = pattern[pos].item()

    # Extract actual attention values for the explanation
    if names:
        indirect_obj = names[0][1]  # First name = indirect object
        subject = names[1][1] if len(names) > 1 else "unknown"

        # Get actual attention to indirect object from Name Mover
        nm_attn = attention_info.get((9, 9), {}).get(indirect_obj, 0.0)
        # Get actual attention to subject from S-Inhibition
        si_attn = attention_info.get((8, 10), {}).get(subject, 0.0)
    else:
        indirect_obj = predicted_word
        subject = "unknown"
        nm_attn = 0.0
        si_attn = 0.0

    # Build NL explanation with ACTUAL computed values
    explanation = f"""
══════════════════════════════════════════════════════════════════════
                    IOI PREDICTION EXPLANATION
══════════════════════════════════════════════════════════════════════

INPUT: "{prompt}"

PREDICTION: "{predicted_word}" (confidence: {confidence:.1%})

WHY THIS PREDICTION?

The model identifies '{predicted_word}' as the indirect object (recipient
of the action) through a multi-step circuit:

1. NAME MOVER HEADS (L9H9, L9H6, L10H0):
   Head L9H9 attends to '{indirect_obj}' with {nm_attn:.1%} attention,
   copying this name to the output position.
   Combined causal importance: {head_importance.get((9,9), 0) + head_importance.get((9,6), 0) + head_importance.get((10,0), 0):.1%}

2. S-INHIBITION HEADS (L8H10, L7H3):
   Head L8H10 attends to '{subject}' with {si_attn:.1%} attention,
   suppressing it to prevent repetition.
   Combined causal importance: {head_importance.get((8,10), 0) + head_importance.get((7,3), 0):.1%}

CAUSAL EVIDENCE:
When we ablate these 6 heads, prediction accuracy drops significantly,
proving they are necessary (not just correlated) for this behavior.
Total circuit coverage: {sum(head_importance.values()):.1%}
══════════════════════════════════════════════════════════════════════
"""
    return explanation, predicted_word, confidence, attention_info


# Generate explanations for all prompts
print("=" * 70)
print("GENERATING NATURAL LANGUAGE EXPLANATIONS")
print("(Novel Contribution: Circuit → Human-Readable)")
print("=" * 70)

for prompt in test_prompts:
    explanation, pred, conf, attn = generate_nl_explanation(prompt, model, head_importance)
    print(explanation)


GENERATING NATURAL LANGUAGE EXPLANATIONS
(Novel Contribution: Circuit → Human-Readable)

══════════════════════════════════════════════════════════════════════
                    IOI PREDICTION EXPLANATION
══════════════════════════════════════════════════════════════════════

INPUT: "When Mary and John went to the store, John gave a drink to"

PREDICTION: "Mary" (confidence: 67.7%)

WHY THIS PREDICTION?

The model identifies 'Mary' as the indirect object (recipient
of the action) through a multi-step circuit:

1. NAME MOVER HEADS (L9H9, L9H6, L10H0):
   Head L9H9 attends to 'Mary' with 66.5% attention,
   copying this name to the output position.
   Combined causal importance: 29.9%

2. S-INHIBITION HEADS (L8H10, L7H3):
   Head L8H10 attends to 'John' with 13.5% attention,
   suppressing it to prevent repetition.
   Combined causal importance: 22.6%

CAUSAL EVIDENCE:
When we ablate these 6 heads, prediction accuracy drops significantly,
proving they are necessary (not just correlated

In [ ]:
# ==============================================================================
# CELL 4: ERASER Faithfulness Evaluation
# ==============================================================================
# Evaluate how faithful our explanations are using ERASER metrics
# Reference: DeYoung et al. 2020 - "ERASER: A Benchmark to Evaluate Rationalized NLP Models"
# This evaluates the CIRCUIT's faithfulness, not the NL text.
# The NL explanation (Cell 3) is a human-readable rendering of the same circuit.
# Faithfulness is measured at the circuit level.

def evaluate_faithfulness(prompt, model, important_heads, k=6):
    """
    Evaluate sufficiency and comprehensiveness of explanations.

    Sufficiency: Do the cited heads account for the model's behavior?
    Comprehensiveness: Do the cited heads capture ALL relevant computation?
    """

    tokens = model.to_tokens(prompt)
    token_strs = model.to_str_tokens(prompt)

    # Find names
    names = []
    for i, tok in enumerate(token_strs):
        tok_clean = tok.strip()
        if (tok_clean and tok_clean[0].isupper() and tok_clean.isalpha() and
            len(tok_clean) > 1 and tok_clean not in NOT_NAMES):
            if tok_clean not in [n[1] for n in names]:
                names.append((i, tok_clean))

    if len(names) < 2:
        return None

    indirect_obj, subject = names[0][1], names[1][1]
    io_tok = model.to_single_token(" " + indirect_obj)
    subj_tok = model.to_single_token(" " + subject)

    # Run with cache
    logits, cache = model.run_with_cache(tokens)
    clean_diff = (logits[0, -1, io_tok] - logits[0, -1, subj_tok]).item()

    # Get logit difference direction
    logit_diff_dir = model.W_U[:, io_tok] - model.W_U[:, subj_tok]

    # Calculate head contributions via direct logit attribution
    head_contributions = {}
    for layer in range(model.cfg.n_layers):
        z = cache[f"blocks.{layer}.attn.hook_z"]
        W_O = model.W_O[layer]
        for head in range(model.cfg.n_heads):
            head_out = z[0, -1, head, :]
            head_contribution = head_out @ W_O[head]
            contribution = (head_contribution @ logit_diff_dir).item()
            head_contributions[(layer, head)] = contribution

    # Get top k heads
    top_heads = sorted(important_heads.items(), key=lambda x: -x[1])[:k]
    top_head_set = set([h[0] for h in top_heads])

    # SUFFICIENCY: Fraction of logit diff from cited heads
    cited_contribution = sum(head_contributions.get(h, 0) for h in top_head_set)
    sufficiency = cited_contribution / clean_diff if clean_diff > 0 else 0

    # COMPREHENSIVENESS: Remove important heads, measure drop
    def remove_important(activation, hook):
        layer = int(hook.name.split('.')[1])
        new_activation = activation.clone()
        for head in range(activation.shape[2]):
            if (layer, head) in top_head_set:
                new_activation[:, :, head, :] = 0
        return new_activation

    hook_names = [f"blocks.{l}.attn.hook_z" for l in range(model.cfg.n_layers)]
    comp_logits = model.run_with_hooks(
        tokens,
        fwd_hooks=[(name, remove_important) for name in hook_names]
    )
    comp_diff = (comp_logits[0, -1, io_tok] - comp_logits[0, -1, subj_tok]).item()
    comprehensiveness = 1 - (comp_diff / clean_diff) if clean_diff > 0 else 0

    return {
        'sufficiency': max(0, min(1, sufficiency)),
        'comprehensiveness': max(0, min(1, comprehensiveness)),
        'clean_logit_diff': clean_diff,
        'indirect_object': indirect_obj,
        'subject': subject
    }


# Run evaluation
print("\n" + "=" * 70)
print("ERASER FAITHFULNESS EVALUATION")
print("Method: Circuit-Based NL Explanations")
print("=" * 70)

all_suff = []
all_comp = []

for prompt in test_prompts:
    result = evaluate_faithfulness(prompt, model, head_importance, k=6)

    if result:
        all_suff.append(result['sufficiency'])
        all_comp.append(result['comprehensiveness'])

        print(f"\nPrompt: ...{prompt[-40:]}")
        print(f"  {result['indirect_object']} vs {result['subject']}")
        print(f"  Sufficiency:       {result['sufficiency']:.1%}")
        print(f"  Comprehensiveness: {result['comprehensiveness']:.1%}")

print("\n" + "=" * 70)
print("PRELIMINARY RESULTS (n=5)")
print("=" * 70)
print(f"  Mean Sufficiency:       {np.mean(all_suff):.1%}")
print(f"  Mean Comprehensiveness: {np.mean(all_comp):.1%}")

f1 = 2 * np.mean(all_suff) * np.mean(all_comp) / (np.mean(all_suff) + np.mean(all_comp)) if (np.mean(all_suff) + np.mean(all_comp)) > 0 else 0
print(f"  Faithfulness F1:        {f1:.1%}")

print("\n⚠️ Note: These are preliminary results with n=5 prompts.")
print("   See 05_expanded_evaluation.ipynb for canonical results (n=50).")



ERASER FAITHFULNESS EVALUATION
Method: Circuit-Based NL Explanations

Prompt: ... went to the store, John gave a drink to
  Mary vs John
  Sufficiency:       100.0%
  Comprehensiveness: 28.4%

Prompt: ...went to the park, Bob handed a flower to
  Alice vs Bob
  Sufficiency:       100.0%
  Comprehensiveness: 24.3%

Prompt: ...went to the office, Tom sent an email to
  Sarah vs Tom
  Sufficiency:       100.0%
  Comprehensiveness: 61.5%

Prompt: ...the restaurant, James passed the menu to
  Emma vs James
  Sufficiency:       100.0%
  Comprehensiveness: 42.5%

Prompt: ...ent to the library, David gave a book to
  Lisa vs David
  Sufficiency:       100.0%
  Comprehensiveness: 20.1%

PRELIMINARY RESULTS (n=5)
  Mean Sufficiency:       100.0%
  Mean Comprehensiveness: 35.3%
  Faithfulness F1:        52.2%

⚠️ Note: These are preliminary results with n=5 prompts.
   See 05_expanded_evaluation.ipynb for canonical results (n=50).


In [ ]:
# ==============================================================================
# CELL 5: Compare With Random Baseline
# ==============================================================================

import random

def evaluate_random_baseline(prompt, model, k=6):
    """Random head selection as baseline."""
    random_heads = {}
    all_heads = [(l, h) for l in range(model.cfg.n_layers) for h in range(model.cfg.n_heads)]
    selected = random.sample(all_heads, k)
    for h in selected:
        random_heads[h] = 1.0 / k
    return evaluate_faithfulness(prompt, model, random_heads, k=k)


print("\n" + "=" * 70)
print("BASELINE COMPARISON")
print("=" * 70)

random_suff = []
random_comp = []

# Run random baseline (average over 5 runs for stability)
for _ in range(5):
    for prompt in test_prompts:
        result = evaluate_random_baseline(prompt, model, k=6)
        if result:
            random_suff.append(result['sufficiency'])
            random_comp.append(result['comprehensiveness'])

random_f1 = 2 * np.mean(random_suff) * np.mean(random_comp) / (np.mean(random_suff) + np.mean(random_comp)) if (np.mean(random_suff) + np.mean(random_comp)) > 0 else 0

print("\n📊 COMPARISON TABLE")
print("-" * 50)
print(f"{'Method':<25} {'Suff':>8} {'Comp':>8} {'F1':>8}")
print("-" * 50)
print(f"{'Circuit-Based (Ours)':<25} {np.mean(all_suff):>7.1%} {np.mean(all_comp):>7.1%} {f1:>7.1%}")
print(f"{'Random Baseline':<25} {np.mean(random_suff):>7.1%} {np.mean(random_comp):>7.1%} {random_f1:>7.1%}")
print("-" * 50)

improvement = ((f1 - random_f1) / random_f1 * 100) if random_f1 > 0 else 0
print(f"\n✅ our method outperforms random by {improvement:.0f}%")



BASELINE COMPARISON

📊 COMPARISON TABLE
--------------------------------------------------
Method                        Suff     Comp       F1
--------------------------------------------------
Circuit-Based (Ours)       100.0%   35.3%   52.2%
Random Baseline             47.8%   22.0%   30.1%
--------------------------------------------------

✅ our method outperforms random by 73%


In [ ]:
# ==============================================================================
# CELL 6: Save Results
# ==============================================================================

from google.colab import drive
import pickle
from datetime import datetime
import os


# Create directory
THESIS_DIR = '/content/drive/MyDrive/thesis'
os.makedirs(f'{THESIS_DIR}/experiments', exist_ok=True)

# Save results
results = {
    'notebook': '03_nl_explanation_generator',
    'method': 'Circuit-Based NL Explanations (Template)',
    'n_prompts': len(test_prompts),
    'your_sufficiency': np.mean(all_suff),
    'your_comprehensiveness': np.mean(all_comp),
    'your_f1': f1,
    'random_sufficiency': np.mean(random_suff),
    'random_comprehensiveness': np.mean(random_comp),
    'random_f1': random_f1,
    'improvement_over_random': improvement,
    'head_importance': head_importance,
    'test_prompts': test_prompts,
    'all_sufficiency_values': all_suff,
    'all_comprehensiveness_values': all_comp,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M')
}

timestamp = datetime.now().strftime('%Y%m%d_%H%M')
path = f'{THESIS_DIR}/experiments/03_nl_explanation_{timestamp}.pkl'

with open(path, 'wb') as f:
    pickle.dump(results, f)

print(f"\n✅ Results saved: {path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

✅ Results saved: /content/drive/MyDrive/thesis/experiments/03_nl_explanation_20260206_1203.pkl


In [ ]:
# ==============================================================================
# CELL 7: Summary
# ==============================================================================

print("\n" + "=" * 70)
print("NOTEBOOK 03 SUMMARY: NL EXPLANATION GENERATOR (RQ2)")
print("=" * 70)

print(f"""
Research Question 2: Can we generate faithful NL explanations?
Answer: YES ✅ (with caveats)

PRELIMINARY RESULTS (n={len(test_prompts)}):
  Sufficiency:       {np.mean(all_suff):.1%}
  Comprehensiveness: {np.mean(all_comp):.1%}
  F1 Score:          {f1:.1%}

vs RANDOM BASELINE:
  Random F1:         {random_f1:.1%}
  Improvement:       +{improvement:.0f}%

INTERPRETATION:
- High sufficiency (100%) = Our cited heads fully explain the behavior
- Lower comprehensiveness = Model has backup/distributed computation
- This is a REAL FINDING about model architecture

""")


NOTEBOOK 03 SUMMARY: NL EXPLANATION GENERATOR (RQ2)

Research Question 2: Can we generate faithful NL explanations?
Answer: YES ✅ (with caveats)

PRELIMINARY RESULTS (n=5):
  Sufficiency:       100.0%
  Comprehensiveness: 35.3%
  F1 Score:          52.2%

vs RANDOM BASELINE:
  Random F1:         30.1%
  Improvement:       +73%

INTERPRETATION:
- High sufficiency (100%) = Our cited heads fully explain the behavior
- Lower comprehensiveness = Model has backup/distributed computation
- This is a REAL FINDING about model architecture


